# ECG Split 1D-CNN Client Side

This code is the server part of ECG split 1D-CNN model for **single** client and a server.

## Import required packages

In [1]:
import os
import struct
import socket
import pickle
import time
from pathlib import Path
from icecream import ic

import h5py
import numpy as np

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam, SGD

print(f'torch version: {torch.__version__}')

torch version: 1.8.1+cu102


## Define ECG dataset class

In [2]:
project_path = Path.cwd().parent.parent
print(f'project_path: {project_path}')

project_path: /home/dk/Desktop/split-learning-1D-HE


In [3]:
# paths to files and directories
data_dir = 'data'  # used to be 'mitdb'
train_name = 'train_ecg.hdf5'
test_name = 'test_ecg.hdf5'
dry_run = True  # load less data (50 examples)

In [4]:
class ECG(Dataset):
    def __init__(self, train=True):
        if train:
            with h5py.File(project_path/data_dir/train_name, 'r') as hdf:
                self.x = hdf['x_train'][:]
                self.y = hdf['y_train'][:]
        else:
            with h5py.File(project_path/data_dir/test_name, 'r') as hdf:
                self.x = hdf['x_test'][:]
                self.y = hdf['y_test'][:]
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return torch.tensor(self.x[idx], dtype=torch.float), torch.tensor(self.y[idx])

### Set batch size

In [5]:
batch_size = 32

## Make train and test dataset batch generator

In [6]:
train_dataset = ECG(train=True)
test_dataset = ECG(train=False)
train_loader = DataLoader(train_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

In [7]:
for i, b in enumerate(train_loader):
    x, label = b
    ic(x.shape, label)
    if i == 1: break

ic| x.shape: torch.Size([32, 1, 128])
    label: tensor([2, 4, 2, 4, 2, 1, 1, 1, 2, 1, 0, 0, 2, 4, 3, 1, 2, 4, 1, 0, 1, 0, 4, 2,
                   4, 3, 0, 1, 2, 4, 4, 2])
ic| x.shape: torch.Size([32, 1, 128])
    label: tensor([0, 4, 3, 4, 2, 0, 1, 4, 1, 0, 3, 4, 2, 4, 1, 0, 2, 0, 0, 2, 0, 1, 1, 0,
                   2, 4, 4, 2, 4, 4, 1, 4])


### Total number of batches

In [8]:
total_batch = len(train_loader)
print(total_batch)

414


## Define ECG client model
Client side has only **2 convolutional layers**.

In [9]:
class EcgClient(nn.Module):
    def __init__(self):
        super(EcgClient, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, 7, padding=3)  # 128 x 16
        self.relu1 = nn.LeakyReLU()
        self.pool1 = nn.MaxPool1d(2)  # 64 x 16
        self.conv2 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
        self.relu2 = nn.LeakyReLU()
        self.pool2 = nn.MaxPool1d(2)  # 32 x 16
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(-1, 32 * 16)
        return x

In [10]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
torch.cuda.get_device_name(0)

'NVIDIA GeForce GTX 1070 Ti'

### Set random seed

In [11]:
seed = 0
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

### Assign intial weight as same as non-split model

In [12]:
ecg_client = EcgClient()
checkpoint = torch.load("init_weight.pth")
ecg_client.conv1.weight.data = checkpoint["conv1.weight"]
ecg_client.conv1.bias.data = checkpoint["conv1.bias"]
ecg_client.conv2.weight.data = checkpoint["conv2.weight"]
ecg_client.conv2.bias.data = checkpoint["conv2.bias"]
ecg_client.to(device)

EcgClient(
  (conv1): Conv1d(1, 16, kernel_size=(7,), stride=(1,), padding=(3,))
  (relu1): LeakyReLU(negative_slope=0.01)
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv1d(16, 16, kernel_size=(5,), stride=(1,), padding=(2,))
  (relu2): LeakyReLU(negative_slope=0.01)
  (pool2): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
)

### Set other hyperparameters in the model
Hyperparameters here should be same with the server side.

In [13]:
epoch = 400
criterion = nn.CrossEntropyLoss()
lr = 0.001
optimizer = Adam(ecg_client.parameters(), lr=lr)

## Socket initialization

### Required socket functions

In [14]:
def send_msg(sock, msg):
    # prefix each message with a 4-byte length in network byte order
    msg = struct.pack('>I', len(msg)) + msg
    sock.sendall(msg)

def recv_msg(sock):
    # read message length and unpack it into an integer
    raw_msglen = recvall(sock, 4)
    if not raw_msglen:
        return None
    msglen = struct.unpack('>I', raw_msglen)[0]
    # read the message data
    return recvall(sock, msglen)

def recvall(sock, n):
    # helper function to receive n bytes or return None if EOF is hit
    data = b''
    while len(data) < n:
        packet = sock.recv(n - len(data))
        if not packet:
            return None
        data += packet
    return data

### Set host address and port number

In [15]:
host = 'localhost'
port = 10080
max_recv = 4096

### Open the client socket

In [16]:
s = socket.socket()
s.connect((host, port))

## Real training process

In [17]:
for e in range(epoch):
    print("Epoch {} - ".format(e+1), end='')
    
    for _, batch in enumerate(train_loader):
        x, label = batch
        x, label = x.to(device), label.to(device)
        optimizer.zero_grad()
        output = ecg_client(x)
        client_output = output.clone().detach().requires_grad_(True)
        msg = {
            'client_output': client_output,
            'label': label
        }
        msg = pickle.dumps(msg)
        send_msg(s, msg)
        msg = recv_msg(s)
        client_grad = pickle.loads(msg)
        output.backward(client_grad)
        optimizer.step()
            
    with torch.no_grad():  # calculate test accuracy
        for _, batch in enumerate(test_loader):
            x, label = batch
            x, label = x.to(device), label.to(device)
            client_output = ecg_client(x)
            msg = {
                'client_output': client_output,
                'label': label
            }
            msg = pickle.dumps(msg)
            send_msg(s, msg)
    
    msg = recv_msg(s)
    train_test_status = pickle.loads(msg)
    print(train_test_status)

Epoch 1 - loss: 1.3244, acc: 59.06% / test_loss: 1.1545, test_acc: 77.38%
Epoch 2 - loss: 1.0868, acc: 83.05% / test_loss: 1.0626, test_acc: 85.08%
Epoch 3 - loss: 1.0519, acc: 85.80% / test_loss: 1.0368, test_acc: 86.90%
Epoch 4 - loss: 1.0411, acc: 86.67% / test_loss: 1.0310, test_acc: 87.67%
Epoch 5 - loss: 1.0351, acc: 87.17% / test_loss: 1.0232, test_acc: 88.29%
Epoch 6 - loss: 1.0326, acc: 87.41% / test_loss: 1.0288, test_acc: 87.70%
Epoch 7 - loss: 1.0279, acc: 87.82% / test_loss: 1.0219, test_acc: 88.40%
Epoch 8 - loss: 1.0250, acc: 88.11% / test_loss: 1.0199, test_acc: 88.53%
Epoch 9 - loss: 1.0253, acc: 87.97% / test_loss: 1.0260, test_acc: 88.04%
Epoch 10 - loss: 1.0234, acc: 88.16% / test_loss: 1.0166, test_acc: 88.80%
Epoch 11 - loss: 1.0207, acc: 88.42% / test_loss: 1.0143, test_acc: 88.97%
Epoch 12 - loss: 1.0202, acc: 88.37% / test_loss: 1.0117, test_acc: 89.26%
Epoch 13 - loss: 1.0194, acc: 88.40% / test_loss: 1.0116, test_acc: 89.25%
Epoch 14 - loss: 1.0170, acc: 88.6

In [18]:
print('Finished Training!')
print('Result is on the server side.')

Finished Training!
Result is on the server side.
